# Basics

## Basic Representation

States and operators in {py:mod}`quimb` are simply dense numpy arrays
or sparse scipy matrices. All functions should directly work with these
but the class {class}`~quimb.core.qarray` is also provided as a very
thin subclass of {class}`numpy.ndarray` with a few helpful methods and
attributes. The {py:func}`~quimb.core.quimbify` function (aliased to
{py:func}`~quimb.core.qu`) can convert between the various representations.

In [1]:
from quimb import *
data = [1, 2j, -3]

Kets are column vectors, i.e. with shape ``(d, 1)``:

In [2]:
qu(data, qtype='ket')

[[ 1.+0.j]
 [ 0.+2.j]
 [-3.+0.j]]

The ``normalized=True`` option can be used to ensure a normalized output.

Bras are row vectors, i.e. with shape ``(1, d)``:

In [3]:
qu(data, qtype='bra')  # also conjugates the data

[[ 1.-0.j  0.-2.j -3.-0.j]]

And operators are square matrices, i.e. have shape ``(d, d)``:

In [4]:
qu(data, qtype='dop')

[[ 1.+0.j  0.-2.j -3.-0.j]
 [ 0.+2.j  4.+0.j  0.-6.j]
 [-3.+0.j  0.+6.j  9.+0.j]]

Which can also be sparse:

In [5]:
qu(data, qtype='dop', sparse=True)

<Compressed Sparse Row sparse matrix of dtype 'complex128'
	with 9 stored elements and shape (3, 3)>

The sparse format can be specified with the `stype` keyword. The partial
function versions of each of the above are also available:

- {func}`~quimb.core.ket`
- {func}`~quimb.core.bra`
- {func}`~quimb.core.dop`
- {func}`~quimb.core.sparse`

:::{note}
If a simple 1d-list is supplied and no `qtype` is given, `'ket'` is
assumed.
:::

## Basic Operations

The 'dagger', or hermitian conjugate, operation is performed with the `.H`
attribute:

In [6]:
psi = 1.0j * bell_state('psi-')
psi

[[ 0.+0.j      ]
 [ 0.+0.707107j]
 [-0.-0.707107j]
 [ 0.+0.j      ]]

In [7]:
psi.H

[[ 0.-0.j        0.-0.707107j -0.+0.707107j  0.-0.j      ]]

This is just the combination of `.conj()` and `.T`, but only available for
{mod}`scipy.sparse` matrices  and {class}`~quimb.core.qarray` s (not
{class}`numpy.ndarray` s).

The product of two quantum objects is the dot or matrix product, which, since
python 3.5, has been overloaded with the `@` symbol. Using it is recommended:

In [8]:
psi = up()
psi

[[1.+0.j]
 [0.+0.j]]

In [9]:
psi.H @ psi  # inner product

[[1.+0.j]]

In [10]:
X = pauli('X')
X @ psi  # act as gate

[[0.+0.j]
 [1.+0.j]]

In [11]:
(psi.H @ X @ psi)  # operator expectation

[[0.+0.j]]

Scalar expectation values might best be computed using the
{func}`~quimb.core.expectation` function (aliased to
{func}`~quimb.core.expec`) which dispatches to accelerated
methods:

In [12]:
expec(psi, psi)

np.float64(1.0)

In [13]:
expec(psi, X)

np.complex128(0j)

Here's an example for a much larger (20 qubit), sparse operator expecation,
which will be automatically parallelized:

In [14]:
psi = rand_ket(2**20)
A = rand_herm(2**20, sparse=True) + speye(2**20)
A

<Compressed Sparse Row sparse matrix of dtype 'complex128'
	with 11534284 stored elements and shape (1048576, 1048576)>

In [15]:
expec(A, psi)  # should be ~ 1

np.float64(1.0004858170636874)

In [16]:
%%timeit
expec(A, psi)

55.1 ms ± 3.82 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Combining Objects - Tensor Products

There are a number of ways to combine states and operators, i.e. tensoring them
together.

Functional form using {func}`~quimb.core.kron`:

```python
>>> kron(psi1, psi2, psi3, ...)
...
```

This can also be done using the `&` overload on `qarray` and scipy matrices:

```python
>>> psi1 & psi2 & psi3
...
```

:::{warning}
When {mod}`quimb` is imported, it monkey patches the otherwise unused
method of `&`/`__and__` of scipy sparse matrices to {func}`~quimb.core.kron`.
:::

Often one wants to sandwich an operator with many identities,
{func}`~quimb.core.ikron` can be used for this:

In [17]:
dims = [2] * 10  # overall space of 10 qubits
X = pauli('X')
IIIXXIIIII = ikron(X, dims, inds=[3, 4])  # act on 4th and 5th spin only
IIIXXIIIII.shape

(1024, 1024)

For more advanced tensor constructions, such as reversing and interleaving
identities within operators {func}`~quimb.core.pkron` can be used:

In [18]:
dims = [2] * 3
XZ = pauli('X') & pauli('Z')
ZIX = pkron(XZ, dims, inds=[2, 0])
ZIX.real.astype(int)

[[ 0  1  0  0  0  0  0  0]
 [ 1  0  0  0  0  0  0  0]
 [ 0  0  0  1  0  0  0  0]
 [ 0  0  1  0  0  0  0  0]
 [ 0  0  0  0  0 -1  0  0]
 [ 0  0  0  0 -1  0  0  0]
 [ 0  0  0  0  0  0  0 -1]
 [ 0  0  0  0  0  0 -1  0]]

`ZIX` would then act with Z on first spin, and X on 3rd.

## Removing Objects - Partial Trace

To remove, or ignore, certain parts of a quantum state the partial trace
function {func}`~quimb.core.partial_trace` (aliased to {func}`~quimb.core.ptr`)
is used. Here, the internal dimensions of a state must be supplied as well as
the indicies of which of these subsystems to *keep*.

For example, if we have a random system of 10 qubits (hilbert space of dimension
`2**10`), and we want just the reduced density matrix describing the first and
last spins:

In [19]:
dims = [2] * 10
D = prod(dims)
psi = rand_ket(D)
rho_ab = ptr(psi, dims, [0, 9])
rho_ab.round(3)  # probably pretty close to identity

[[ 0.279+0.j     0.003-0.005j  0.012+0.009j  0.018-0.009j]
 [ 0.003+0.005j  0.239+0.j    -0.005+0.007j -0.005-0.002j]
 [ 0.012-0.009j -0.005-0.007j  0.24 +0.j    -0.015-0.014j]
 [ 0.018+0.009j -0.005+0.002j -0.015+0.014j  0.242+0.j   ]]

{func}`~quimb.core.partial_trace` accepts dense or sparse, operators or vectors.